#### Squared differences estimation

Out is the output of our model and it returns a set of 6 arrays representing the curve of infection for the age groups.

Time, t, has its unit step representing a fortnight, thus we will select only the odd entries of the array and compute the first finite differences for each age class to get the number of new cases observed monthly. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
age_class = ["<1", "1-4", "5-9", "10-14", "15-19", "20-29", "30+"]

#### Vector $\beta$ (= force of infection) to estimate

In [113]:
#beta_range = np.arange(3, 15.2, step = 0.2)

In [1]:
import itertools
#TOO BIG
#beta_combinations = list(itertools.product(beta_range, repeat = 6))

In [ ]:
list(beta_range)

In [66]:
beta = np.eye(6)*[1, 2, 3, 4, 5, 6]

In [4]:
#Select disease
d = "Measles"

#monthly data
monthly_data = pd.read_csv( d +'_data_processed.csv')
time_date = monthly_data.iloc[:,0]

#cumulative monthly data
ecdc_disease_noTime = monthly_data.drop("Time", axis=1)
cumulative_data = ecdc_disease_noTime.apply(lambda x: np.cumsum(x))

In [9]:
len(time_date)*2

504

###### Based on the monthly / cumulative data

In [ ]:
def f_ssq(beta_vec):
    
    beta = np.eye(7)*beta_vec
    mdl = m.SIRVModel(para, lambda t: beta)
    new_ic = np.hstack([0.95*(1./7)*np.ones(2), np.zeros(5), 0.05*(1./7)*np.ones(2), np.zeros(5), np.zeros(7), V_ic])
    out, t = mdl.run(new_ic ,t_max = 19,method="RK45", eval_per_year=24)
    
    #Something to do with this monthly cases
    
    monthly_OUT = out[7:(7*2),1::2]
    groups = monthly_OUT.shape[0]
    ssq_diff = 0
    for ii in range(groups):
    Inf_group = OUT[ii,:]
    cum_infected_sim = Inf_group[1::2]
    #monthly data
    monthly_infected_sim = np.diff(cum_infected_sim)
    #adding the squared diff
    ssq_diff += (monthly_infected_sim - monthly_data[age_class[ii]])**2

In [ ]:
monthly_OUT = out[7:(7*2),1::2]

groups = monthly_OUT.shape[0]
ssq_diff = 0
for ii in range(groups):
    Inf_group = OUT[ii,:]
    cum_infected_sim = Inf_group[1::2]
    #monthly data
    monthly_infected_sim = np.diff(cum_infected_sim)
    #adding the squared diff
    ssq_diff += (monthly_infected_sim - monthly_data[age_class[ii]])**2

In [ ]:
scipy.optimize.fmin(f_ssq, x0 = beta = [0.98, 0.9, 0.85, 0.8, 0.7, 0.7, 0.7] , args=(), xtol=0.0001, ftol=0.0001, maxiter=None, maxfun=None, full_output=0, disp=1, retall=0, callback=None)[source]